In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms

In [3]:
# transform raw image to tensor

transformations = transforms.Compose([transforms.ToTensor()])

In [4]:
# Load CIFAR-10

train_set = datasets.CIFAR10(root='./data', train=True, transform=transformations, download=True)
test_set  = datasets.CIFAR10(root='./data', train=False, transform=transformations, download=True)

Files already downloaded and verified


In [5]:
# Load MNIST

# train_set = datasets.MNIST(root='./data', train=True, transform=transformations, download=True)
# test_set  = datasets.MNIST(root='./data', train=False, transform=transformations, download=True)

In [6]:
# number of images to process per batch

batch_size = 64

In [7]:
# load train set

train_set_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=batch_size,
    shuffle=False
)

In [8]:
# load test set

test_set_loader = torch.utils.data.DataLoader(
    dataset=test_set,
    batch_size=batch_size,
    shuffle=False
)

<center><img src="img/lenet.png"></center>

In [9]:
class LeNet(nn.Module):
    
    """
    input - 3x32x32
    C1 - convolution 6@28x28 (5x5 kernel)
    ReLU activation
    S2 - max pooling 6@14x14 (2x2 kernel, stride 2)

    C3 - convolution 16@10x10 (5x5 kernel)
    ReLU activation
    S4 - max pooling 16@5x5 (2x2 kernel, stride 2)

    C5 - convolution 120@1x1 (5x5 kernel)
    
    F6 - fully-connected layear, 120 neurons
    F7 - fully-connected layear, 84 neurons
    ReLU activation
    
    F8 - 10 neurons (output)
    """
        
    def __init__(self):
        
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, kernel_size=(5, 5))
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5, 5))
        
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        
        out = out.view(out.size(0), -1)
        
        out = F.relu(self.fc1(out))
        
        out = F.relu(self.fc2(out))
        
        out = self.fc3(out)
        
        return out

In [10]:
model = LeNet()

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [12]:
criterion = nn.CrossEntropyLoss()

In [13]:
n_epochs = 10

for epoch in range(1, n_epochs+1):

    # swith to train mode
    model = model.train()
    
    # global statistics
    running_train_loss = 0.
    
    for batch_idx, (inputs, targets) in enumerate(train_set_loader, start=1):

        # convert tensor to autograd.Variable
        inputs, targets = Variable(inputs), Variable(targets)

        # forward pass
        predictions = model.forward(inputs)

        # calculate loss - cross-entropy
        loss = criterion(predictions, targets)
        
        # accumulate loss
        running_train_loss += loss.item()

        # compute gradients
        loss.backward()
        
        # update weights based on calculated gradients
        optimizer.step()

        # clear all gradients after weights are updated
        optimizer.zero_grad()

    # print epoch statistics    
    epoch_train_loss = running_train_loss / len(train_set_loader.dataset)
    print('=> epoch: %d, train loss: %.6f' % (epoch, epoch_train_loss))
        
    # switch to evaluation/test mode
    model = model.eval()

    # global statistics
    running_test_loss = 0.
    running_test_corrects = 0.

    # disable calculation of gradients
    with torch.no_grad():

        for batch_idx, (inputs, targets) in enumerate(test_set_loader, start=1):

            # convert tensor to autograd.Variable
            inputs, targets = Variable(inputs), Variable(targets)
            
            # forward pass - make predictions
            predictions = model.forward(inputs)

            # calculate loss
            loss = criterion(predictions, targets)

            # convert probability to a predicted label
            _, prediction_labels = torch.max(predictions.data, 1)
            
            # calculate how many digits in the batch where predicted correctly
            running_test_corrects += torch.sum(prediction_labels == targets.data).item()

            # accumulate loss
            running_test_loss += loss.item()

    # print epoch statistics    
    epoch_test_loss = running_test_loss / len(test_set_loader.dataset)
    epoch_test_accuracy = running_test_corrects / len(test_set_loader.dataset)
    print('=> epoch: %d, test loss: %.6f, accuracy: %.3f \n' % (epoch, epoch_test_loss, epoch_test_accuracy))

=> epoch: 1, train loss: 0.031222
=> epoch: 1, test loss: 0.028430, accuracy: 0.339 

=> epoch: 2, train loss: 0.024183
=> epoch: 2, test loss: 0.023628, accuracy: 0.455 

=> epoch: 3, train loss: 0.021503
=> epoch: 3, test loss: 0.021061, accuracy: 0.519 

=> epoch: 4, train loss: 0.019887
=> epoch: 4, test loss: 0.019844, accuracy: 0.551 

=> epoch: 5, train loss: 0.018685
=> epoch: 5, test loss: 0.019313, accuracy: 0.564 

=> epoch: 6, train loss: 0.017668
=> epoch: 6, test loss: 0.019354, accuracy: 0.567 

=> epoch: 7, train loss: 0.016801
=> epoch: 7, test loss: 0.019560, accuracy: 0.577 

=> epoch: 8, train loss: 0.016058
=> epoch: 8, test loss: 0.019720, accuracy: 0.577 

=> epoch: 9, train loss: 0.015613
=> epoch: 9, test loss: 0.020277, accuracy: 0.572 

=> epoch: 10, train loss: 0.015212
=> epoch: 10, test loss: 0.020396, accuracy: 0.564 

